In [1]:
import torch
import torch.nn.functional as F

class VPLoss:
    def __call__(self, model, x_0, t, noise):
        x_t = self.q_sample(x_0, t, noise)
        pred_noise = model(x_t, t)
        return F.mse_loss(pred_noise, noise) # recheck the loss function

    def q_sample(self, x_0, t, noise):
        # Apply forward process to add noise
        # return x_t
        pass


In [ ]:
# def c_skip(sigma): return 1 / (sigma**2 + 1)**0.5
# def c_out(sigma): return sigma / (sigma**2 + 1)**0.5
# def c_in(sigma): return 1 / (sigma**2 + 1)**0.5
# def c_noise(sigma): return log(sigma)  # or some scaled version


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

model = UNetDDPMpp().to(device)
denoiser = Denoiser(model)

optimizer = optim.Adam(model.parameters(), lr=2e-4)

mse_loss = nn.MSELoss()

beta_d = 19.9
beta_min = 0.1
epsilon_t = 1e-5

transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
loader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2)

epochs = 50
for epoch in range(epochs):
    model.train()
    pbar = tqdm(loader)
    total_loss = 0

    for x, _ in pbar:
        x = x.to(device)
        B = x.size(0)

        t = torch.rand(B, device=x.device) * (1.0 - epsilon_t) + epsilon_t

        sigma = torch.sqrt(torch.exp(0.5 * beta_d * t ** 2 + beta_min * t) - 1)
        sigma = sigma.view(-1, 1, 1, 1)

        noise = torch.randn_like(x)
        x_noisy = x + sigma * noise

        pred = denoiser(x_noisy, sigma.view(-1))  # sigma input to Denoiser

        loss = (1 / (sigma ** 2)) * (pred - noise) ** 2
        loss = loss.mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_description(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")

    print(f"[Epoch {epoch+1}] Avg Loss: {total_loss / len(loader):.4f}")


NameError: name 'UNetDDPMpp' is not defined